<a href="https://colab.research.google.com/github/elorie-bernard-lacroix/SmartStudy/blob/main/SmartStudy/notebooks/3_23_gpt_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tabpfn
!pip install bayesian-optimization
!pip install sklearn
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch


In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Loading Dataset
data = pd.read_csv("/content/drive/MyDrive/ECE324_Project/Model/processed.csv")
data.info()
features = ['Age', 'Gender', 'StudyTimeWeekly', 'Absences', 'Extracurricular',
            'Sports', 'Music', 'Volunteering', 'ParentalInfluence', 'TutoringEffect']

X = data[features]
y = data['GPA']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, y, random_state=42)

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2392 entries, 0 to 2391
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                2392 non-null   int64  
 1   Gender             2392 non-null   int64  
 2   StudyTimeWeekly    2392 non-null   float64
 3   Absences           2392 non-null   int64  
 4   Extracurricular    2392 non-null   int64  
 5   Sports             2392 non-null   int64  
 6   Music              2392 non-null   int64  
 7   Volunteering       2392 non-null   int64  
 8   GPA                2392 non-null   float64
 9   ParentalInfluence  2392 non-null   int64  
 10  TutoringEffect     2392 non-null   float64
dtypes: float64(3), int64(8)
memory usage: 205.7 KB


In [5]:
!pip install tqdm

## Train the model

In [6]:
from tabpfn import TabPFNRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Train and predict TabPFN
reg = TabPFNRegressor(random_state=42)
reg.fit(X_train, Y_train)
Y3_pred = reg.predict(X_test)

# evaluation
print('Mean Squared Error:', mean_squared_error(Y_test, Y3_pred))
print('Mean Absolute Error:', mean_absolute_error(Y_test, Y3_pred))

/usr/local/lib/python3.11/dist-packages/tabpfn/base.py:101: UserWarning: Downloading model to /root/.cache/tabpfn/tabpfn-v2-regressor.ckpt.
  model, bardist, config_ = load_model_criterion_config(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tabpfn-v2-regressor.ckpt:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

Mean Squared Error: 0.05842898892721607
Mean Absolute Error: 0.188859123829388


 ## Take Raw User Input and Engineer Features

In [7]:
#get users input
user_raw = {
    'Age': int(input("Age: ")),
    'Gender': int(input("Gender (0=Male, 1=Female): ")),
    'StudyTimeWeekly': float(input("Study Time Weekly: ")),
    'Absences': int(input("Absences: ")),
    'Extracurricular': int(input("Extracurricular (0 or 1): ")),
    'Sports': int(input("Sports (0 or 1): ")),
    'Music': int(input("Music (0 or 1): ")),
    'Volunteering': int(input("Volunteering (0 or 1): ")),
    'Tutoring': int(input("Tutoring (0 or 1): ")),
    'ParentalSupport': int(input("Parental Support (0-4): ")),
    'ParentalEducation': int(input("Parental Education (0-3): "))
}

# ask user for their targeted gpa
target_gpa = float(input("Target GPA (e.g., 3.5): "))

# inculude feature engineering
user_engineered = {
    'Age': user_raw['Age'],
    'Gender': user_raw['Gender'],
    'StudyTimeWeekly': user_raw['StudyTimeWeekly'],
    'Absences': user_raw['Absences'],
    'Extracurricular': user_raw['Extracurricular'],
    'Sports': user_raw['Sports'],
    'Music': user_raw['Music'],
    'Volunteering': user_raw['Volunteering'],
    'ParentalInfluence': user_raw['ParentalSupport'] * user_raw['ParentalEducation'],
    'TutoringEffect': user_raw['Tutoring'] * user_raw['StudyTimeWeekly']
}

Age: 16
Gender (0=Male, 1=Female): 1
Study Time Weekly: 8
Absences: 2
Extracurricular (0 or 1): 1
Sports (0 or 1): 0
Music (0 or 1): 0
Volunteering (0 or 1): 1
Tutoring (0 or 1): 0
Parental Support (0-4): 1
Parental Education (0-3): 2
Target GPA (e.g., 3.5): 3.7


#### Predict current GPA

In [8]:
user_df = pd.DataFrame([user_engineered])
user_scaled = scaler.transform(user_df)
predicted_gpa = reg.predict(user_scaled)[0]
print("📘 Current Predicted GPA:", predicted_gpa)

📘 Current Predicted GPA: 2.9679651


## Run Bayesian Optimization

In [9]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 4.5 MB/s eta 0:00:00


In [14]:
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args

space = [
    Real(0.0, 20.0, name='StudyTimeWeekly'),
    Integer(0, 29, name='Absences'),
    Categorical([0, 1], name='Tutoring'),
    Integer(0, 4, name='ParentalSupport'),
    Categorical([0, 1], name='Extracurricular'),
    Categorical([0, 1], name='Sports'),
    Categorical([0, 1], name='Music'),
    Categorical([0, 1], name='Volunteering'),
    Integer(0, 3, name='ParentalEducation')
]

@use_named_args(space)
def objective(**params):
    tutoring_effect = params['Tutoring'] * params['StudyTimeWeekly']
    parental_influence = params['ParentalSupport'] * params['ParentalEducation']

    engineered_input = {
        'Age': user_raw['Age'],
        'Gender': user_raw['Gender'],
        'StudyTimeWeekly': params['StudyTimeWeekly'],
        'Absences': params['Absences'],
        'Extracurricular': params['Extracurricular'],
        'Sports': params['Sports'],
        'Music': params['Music'],
        'Volunteering': params['Volunteering'],
        'ParentalInfluence': parental_influence,
        'TutoringEffect': tutoring_effect
    }

    input_df = pd.DataFrame([engineered_input])
    input_scaled = scaler.transform(input_df)
    pred = reg.predict(input_scaled)[0]
    return abs(pred - target_gpa)

res = gp_minimize(objective, space, n_calls=50, random_state=0)

optimized_params = dict(zip(
    ['StudyTimeWeekly', 'Absences', 'Tutoring', 'ParentalSupport',
     'Extracurricular', 'Sports', 'Music', 'Volunteering', 'ParentalEducation'],
    res.x
))

optimized_habits = {
    'Age': user_raw['Age'],
    'Gender': user_raw['Gender'],
    'StudyTimeWeekly': optimized_params['StudyTimeWeekly'],
    'Absences': optimized_params['Absences'],
    'Extracurricular': optimized_params['Extracurricular'],
    'Sports': optimized_params['Sports'],
    'Music': optimized_params['Music'],
    'Volunteering': optimized_params['Volunteering'],
    'ParentalInfluence': optimized_params['ParentalSupport'] * optimized_params['ParentalEducation'],
    'TutoringEffect': optimized_params['Tutoring'] * optimized_params['StudyTimeWeekly']
}

## Text Generation using GPT 4

In [15]:
!pip install openai

In [16]:
import openai
import getpass

api_key = getpass.getpass("Enter OpenAI API key: ")

Enter OpenAI API key: ··········


In [17]:
client = openai.OpenAI(api_key=api_key)


def generate_recommendations_gpt4(current_habits, optimized_habits, target_gpa):
    prompt = f"""
Student is aiming to improve GPA to {target_gpa}.

Current Habits:
- Study Time: {current_habits['StudyTimeWeekly']} hrs
- Absences: {current_habits['Absences']}
- Extracurricular: {current_habits['Extracurricular']}
- Sports: {current_habits['Sports']}
- Music: {current_habits['Music']}
- Volunteering: {current_habits['Volunteering']}
- Parental Influence Score: {current_habits['ParentalInfluence']}
- Tutoring Effect Score: {current_habits['TutoringEffect']}

Recommended Optimized Habits:
- Study Time: {optimized_habits['StudyTimeWeekly']} hrs
- Absences: {optimized_habits['Absences']}
- Extracurricular: {optimized_habits['Extracurricular']}
- Sports: {optimized_habits['Sports']}
- Music: {optimized_habits['Music']}
- Volunteering: {optimized_habits['Volunteering']}
- Parental Influence Score: {optimized_habits['ParentalInfluence']}
- Tutoring Effect Score: {optimized_habits['TutoringEffect']}

As an academic advisor, explain how these changes will help improve the student’s GPA. Also provide personalized and motivational advice to help them stay on track.
"""

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=600
    )
    return response.choices[0].message.content

    #give an exmaple

In [18]:
recommendation_text = generate_recommendations_gpt4(user_engineered, optimized_habits, target_gpa)
print("\n📘 Personalized Study Plan:\n")
print(recommendation_text)


📘 Personalized Study Plan:

The recommended optimized habits are tailored to provide a well-rounded growth for the student while improving the GPA.

Increasing study time to 20 hours per week will allow for a more robust understanding of the course material, leading to better exam scores and overall grades. It's important, though, to ensure these study hours are focused, efficient, and well-organized.

Absences are increased to 6 which might seem counterintuitive. However, it's important to remember that mental health and wellness are crucial for academic performance. Occasional breaks can prevent burnout and help maintain focus and motivation in the long term.

Maintaining extracurricular activity, and adding sports and music, will contribute to a balanced lifestyle. Sports often help with stress management, team skills, and maintaining a healthy lifestyle. Music can stimulate the brain, relieve stress, and provide a creative outlet.

Reducing volunteering time will provide more hour

# GUI DEMO